In [1]:
import numpy as np
import pandas as pd
import glob
import re
from tqdm import tqdm
import os.path
import collections
from collections import defaultdict
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px


In [2]:

folder_prompt_ = "../../../data/OpenAI/PromptsAnalysesData/"

file = "caribbean_df_orig_GT.csv"
df_caribbean_GT = pd.read_csv(F"{folder_prompt_}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')

file = "caribbean_df_orig_ChatGPT.csv"
df_caribbean_ChatGPT = pd.read_csv(F"{folder_prompt_}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')

In [3]:
# df_caribbean_GT = df_caribbean_GT.droplevel(0, axis=1) 
# df_caribbean_ChatGPT = df_caribbean_ChatGPT.droplevel(0, axis=1) 

# df_caribbean_GT.columns = df_caribbean_GT.columns.get_level_values(0)
# df_caribbean_ChatGPT.columns = df_caribbean_ChatGPT.columns.get_level_values(0)

# df_caribbean_GT.columns = [' '.join(col) for col in df_caribbean_GT.columns.values]
# df_caribbean_ChatGPT.columns = [' '.join(col) for col in df_caribbean_ChatGPT.columns.values]

In [4]:
# df_caribbean_ChatGPT.compare(df_caribbean_GT)
# df_caribbean_ChatGPT.compare(df_caribbean_GT,
#                         # keep_shape=True,
#                         keep_equal=False,
#                         )

df = df_caribbean_GT.subtract(df_caribbean_ChatGPT)
df = df.fillna("Fail")
df = df.applymap(lambda x: 'Correct' if isinstance(x, (int, float)) and x == 0 else x)
df = df.applymap(lambda x: 'Incorrect' if isinstance(x, (int, float)) and x == -1 else x)
df = df.applymap(lambda x: 'Incorrect' if isinstance(x, (int, float)) and x == 1 else x)

In [5]:
df

Life form            Leaf position  \
                            liana       tree     alternate   
Species                                                      
Bursera karsteniana       Correct    Correct          Fail   
Metopium brownei          Correct  Incorrect          Fail   
Handroanthus billbergii   Correct    Correct          Fail   
Avicennia germinans       Correct    Correct          Fail   
Bourreria succulenta      Correct    Correct     Incorrect   

                                                                            \
                        alternate, opposite opposite opposite, whorls of 3   
Species                                                                      
Bursera karsteniana                    Fail     Fail                  Fail   
Metopium brownei                       Fail     Fail                  Fail   
Handroanthus billbergii                Fail     Fail                  Fail   
Avicennia germinans                    Fail     Fail                  Fail   
Bourreria succulenta                Correct  Correct               Correct   

                                                         Leaf composition  \
                        opposite, whorls of 3, alternate        3 palmate   
Species                                                                     
Bursera karsteniana                                 Fail             Fail   
Metopium brownei                                    Fail             Fail   
Handroanthus billbergii                             Fail          Correct   
Avicennia germinans                                 Fail          Correct   
Bourreria succulenta                             Correct             Fail   

                                                 ... Aril colour              \
                        3-5 palmate 3-5 pinnate  ...      orange   red white   
Species                                          ...                           
Bursera karsteniana            Fail        Fail  ...        Fail  Fail  Fail   
Metopium brownei               Fail        Fail  ...        Fail  Fail  Fail   
Handroanthus billbergii     Correct   Incorrect  ...        Fail  Fail  Fail   
Avicennia germinans         Correct     Correct  ...        Fail  Fail  Fail   
Bourreria succulenta           Fail        Fail  ...        Fail  Fail  Fail   

                                    Seed colour                        \
                        yellow-geen       black      brown      green   
Species                                                                 
Bursera karsteniana            Fail        Fail       Fail       Fail   
Metopium brownei               Fail     Correct  Incorrect    Correct   
Handroanthus billbergii        Fail        Fail       Fail       Fail   
Avicennia germinans            Fail   Incorrect  Incorrect  Incorrect   
Bourreria succulenta           Fail        Fail       Fail       Fail   

                                                        
                            grey      white    whitish  
Species                                                 
Bursera karsteniana         Fail       Fail       Fail  
Metopium brownei         Correct    Correct    Correct  
Handroanthus billbergii     Fail       Fail       Fail  
Avicennia germinans      Correct  Incorrect  Incorrect  
Bourreria succulenta        Fail       Fail       Fail  

[5 rows x 202 columns]

### Plotting

In [6]:
group_labels = [trait for (trait, _) in df.columns]
sub_labels = [sublabel.capitalize() for (_, sublabel) in df.columns]
number_labels = [number for number, (_, _) in enumerate(df.columns)]

correct_lst = []
incorrect_lst = []
fail_lst = []
for column in df.columns:

    # Correct
    value = df[column].str.count("Correct").sum() # Cannot sum directly
    correct_lst.append(value)

        # Correct
    value = df[column].str.count("Incorrect").sum() # Cannot sum directly
    incorrect_lst.append(value)

        # Correct
    value = df[column].str.count("Fail").sum() # Cannot sum directly
    fail_lst.append(value)

total = len(df_caribbean_ChatGPT.index)

correct_lst = np.array(correct_lst)
incorrect_lst = np.array(incorrect_lst)
fail_lst = np.array(fail_lst)

correct_lst = correct_lst / total * 100
incorrect_lst = incorrect_lst / total * 100
fail_lst = fail_lst / total * 100



In [7]:
fig = go.Figure()

predictions = [
    "Correct",
    "Incorrect",
    "No Information Found"
]

colors = [
    "green",
    "red",
    "blue"
]

lsts = [
    correct_lst,
    incorrect_lst,
    fail_lst
]

x = [
    group_labels,
    number_labels
]

for (prediction, color, lst) in zip(predictions, colors, lsts):
    fig.add_bar(
        name=prediction,
        x=x,
        y=lst,
        marker_color=color,
        constraintext='outside',
        hovertext=sub_labels,
        textposition='auto'
        )
    
fig.update_layout(
    # uniformtext_minsize=8, uniformtext_mode='hide',
    title='ChatGPT Predictions',
    # xaxis_tickfont_size=6,
    yaxis=dict(
        title='Percentage',
        titlefont_size=16,
        # tickfont_size=3,
    ),

    barmode='relative',
    bargap=0.0, # gap between bars of adjacent location coordinates.
    bargroupgap=0.00, # gap between bars of the same location coordinate.
    
)

fig.show()


folder_figures = "../../../reports/figures/"
file_name = "PxPlot_HTML_BarchartPredictions.html"
fig.write_html(F"{folder_figures}{file_name}")
file_name = "PxPlot_PDF_BarchartPredictions.pdf"
fig.write_image(F"{folder_figures}{file_name}")